In [23]:
# python
import pandas as pd
import numpy as np
import scanpy as sc
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.autograd import Variable
from PIL import Image
from sklearn.model_selection import train_test_split
import argparse
import os
import SIGNET

In [5]:
data = sc.read_h5ad('/rd2/user/xiacr/sle/final/seurat/b_cell/03-b_cell_anno_filter_harm.h5ad')
# data = data.transpose()
tf_gene = pd.read_table('./hs_hgnc_tfs.txt',header=None)
print("data loaded!")
print(data)
# Quality control
sc.pp.filter_genes(data, min_cells=6)
# mouse data: mt-, human data: MT-
mito = 'MT-'
mito_genes = data.var_names.str.startswith(mito)
data.var['mito'] = mito_genes
qc = sc.pp.calculate_qc_metrics(data, qc_vars=['mito'])
cell_qc_dataframe = qc[0]
gene_qc_dataframe = qc[1]
data.obs["n_genes"] = cell_qc_dataframe['n_genes_by_counts']
data.obs["n_counts"] = cell_qc_dataframe['total_counts']
data.obs["percent_mito"] = cell_qc_dataframe['pct_counts_mito']
data = data[data.obs.n_genes < 9000, :]
data = data[data.obs.percent_mito < 20, :]
# raw_data = pd.DataFrame(data.X.transpose().copy())

data loaded!
AnnData object with n_obs × n_vars = 22883 × 23322
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'group', 'treatment', 'pair', 'percent_mito', 'percent_ribo', 'S.Score', 'G2M.Score', 'Phase', 'old.ident', 'RNA_snn_res.0.8', 'RNA_snn_res.1', 'seurat_clusters', 'main_type', 'RNA_snn_res.0.6', 'RNA_snn_res.1.2', 'subtype'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable'
    obsm: 'X_harmony', 'X_pca', 'X_umap'


In [12]:
data

View of AnnData object with n_obs × n_vars = 22883 × 17490
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'group', 'treatment', 'pair', 'percent_mito', 'percent_ribo', 'S.Score', 'G2M.Score', 'Phase', 'old.ident', 'RNA_snn_res.0.8', 'RNA_snn_res.1', 'seurat_clusters', 'main_type', 'RNA_snn_res.0.6', 'RNA_snn_res.1.2', 'subtype', 'n_genes', 'n_counts'
    var: 'vst.mean', 'vst.variance', 'vst.variance.expected', 'vst.variance.standardized', 'vst.variable', 'n_cells', 'mito'
    obsm: 'X_harmony', 'X_pca', 'X_umap'

In [21]:
raw_data = pd.DataFrame(data.X.copy().todense().transpose())
col_name = data.obs_names # cell
row_name = data.var_names # gene
raw_data.index = row_name
raw_data.columns = col_name

In [26]:
sc.pp.normalize_total(data, target_sum=1e4)
sc.pp.log1p(data)
sc.pp.highly_variable_genes(data, n_top_genes = 5000)
gene_list = data.var.highly_variable
gene_list = gene_list.index[gene_list == True]
gene_list = gene_list.to_list()
raw_data_fe = raw_data.loc[gene_list]
list1 = tf_gene[0].to_list()
list2 = raw_data_fe.index
list5 = raw_data.index
list3 = [x for x in list1 if x in list2]
list4 = [x for x in list2 if x not in list1]
raw_data_tf = raw_data.loc[list3, ]
raw_data_ntf = raw_data_fe.loc[list4, ]
gene_ntf = raw_data_ntf.index
gene_tf = raw_data_tf.index
HL_estimator = []
thresholds = []
data_ntf_binary = raw_data_ntf
threshold_index = raw_data_ntf.shape[1]*(raw_data_ntf.shape[1]-1)/4
print("Binarization Begin!")


Binarization Begin!


In [ ]:
for i in range(raw_data_ntf.shape[0]):
    gene_expr = raw_data_ntf.loc[gene_ntf[i], ]
    marker = [0 for x in range(int(gene_expr.max())+1)]
    record = []
    value = []
    for j in range(gene_expr.__len__()):
        marker[int(gene_expr[j])] += 1
    for j in range(int(gene_expr.max())+1):
        if marker[j] != 0:
            value.append(j)
            record.append(marker[j])
    value_calculation = []
    record_calculation = []
    for j in range(int(value.__len__())):
        for k in range(j, value.__len__()):
            value_calculation.append((value[j]+value[k])/2)
            if j == k:
                record_calculation.append(record[j]*(record[j]-1)/2)
            else:
                record_calculation.append(record[j]*record[k])
    test = {'index': value_calculation, 'number': record_calculation}
    HL_estimator = pd.DataFrame(test)
    HL_estimator_new = HL_estimator.sort_values(by=['index'],ascending=True)
    thresin = 0
    for j in range(record_calculation.__len__()):
        thresin += HL_estimator_new.iat[j, 1]
        if thresin >= threshold_index:
            threshold = HL_estimator_new.iat[j, 0]
            for k in range(data_ntf_binary.shape[1]):
                if data_ntf_binary.iat[i, k] > threshold:
                    data_ntf_binary.iat[i, k] = 1
                else:
                    data_ntf_binary.iat[i, k] = 0
            thresholds.append(HL_estimator_new.iat[j, 0])
            break